In [119]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from finvizfinance.screener.overview import Overview
import alpaca_trade_api as tradeapi
import datetime

%matplotlib inline

In [120]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")
print(api_key)

f147b186a1504298a5fca6370fdcfe78


In [121]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [122]:
foverview = Overview()

filters_dict = {
    'Performance':'Today Up',
    'Price':"Under $2", 
    'Relative Volume': "Over 5"
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(8)

,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
Ticker,,,,,,,,,
CYRN,CYREN Ltd.,Technology,Software - Infrastructure,USA,67060000.0,None,1.24,0.0081,7089576.0
DNN,Denison Mines Corp.,Energy,Uranium,Canada,712000000.0,None,1.11,0.0472,152474141.0
LKCO,Luokung Technology Corp.,Communication Services,Internet Content & Information,China,361980000.0,None,1.59,0.3250,55653826.0
RAVE,"RAVE Restaurant Group, Inc.",Consumer Cyclical,Restaurants,USA,31380000.0,None,1.70,0.0303,11882305.0


In [123]:
def get_sentiment(company):
    stock_headlines = ( newsapi.get_everything(
    q=company,
    language='en',
    sort_by='relevancy'))

    sentiments = []

    for articles in stock_headlines['articles']:
        try:
            text = articles['content']
            results = analyzer.polarity_scores(text)
            compound = results['compound']
            pos = results['pos']
            neu = results['neu']
            neg = results['neg'] 

            #put data into sentiments
            sentiments.append({
                'text':text,
                'Compound':compound,
                'Positive':pos,
                'Negative':neg,
                'Neutral':neu})

        except AttributeError:
            pass
    fininal_sentiments = pd.DataFrame(sentiments)
    if not fininal_sentiments.empty:
        return(fininal_sentiments["Positive"].mean())
    else:
        return(0)

In [124]:
ticker_sentiment = [] 
for ticker in tickers.index: 
    sentiment = get_sentiment(tickers['Company'][ticker])
    print(sentiment)
    if float(sentiment) >= 0.1:
        ticker_sentiment.append(ticker)
        print(f"{ticker} does meets our sentiment requirement")
    else:
        print(f"{ticker} does not meet our sentiment requirement")

0.04883333333333333
CYRN does not meet our sentiment requirement
0.10471428571428572
DNN does meets our sentiment requirement
0.050111111111111106
LKCO does not meet our sentiment requirement
0.125
RAVE does meets our sentiment requirement


In [125]:
### Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [128]:
#Set time frame for reporting stocks
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 2) 
start_time = tomorrow.strftime("%b %d %Y") + " 10:00AM"
next_day = today + datetime.timedelta(days = 1) 
end_time = next_day.strftime("%b %d %Y") + " 1:00PM"

date_from = datetime.datetime.strptime(
    start_time, '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime(
    end_time, '%b %d %Y %I:%M%p')

In [131]:
# Set timeframe to '1D'
timeframe = "1D"

#Set timeframe for last 2 years
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp(next_day, tz="America/New_York").isoformat()

#Get ticker data
df_ticker = api.get_barset(
    ticker_sentiment,
    timeframe,
    start=start_date,
    end=end_date
).df

df_ticker.dropna(inplace=True)
df_ticker.tail()

DNN                                    RAVE  \
                             open   high     low   close     volume  open   
time                                                                        
2021-02-08 00:00:00-05:00  0.8379  0.878  0.7990  0.8522   10785141  1.25   
2021-02-09 00:00:00-05:00  0.9459  0.950  0.8953  0.9255   12544068  1.29   
2021-02-10 00:00:00-05:00  1.0400  1.040  0.9000  0.9872   18898232  1.40   
2021-02-11 00:00:00-05:00  0.9713  1.270  0.9524  1.0400  131220760  1.30   
2021-02-12 00:00:00-05:00  1.3300  1.340  1.0700  1.1100  132257543  1.47   

                                                         
                           high   low   close    volume  
time                                                     
2021-02-08 00:00:00-05:00  1.33  1.22  1.2800    806829  
2021-02-09 00:00:00-05:00  1.45  1.25  1.3200   1862123  
2021-02-10 00:00:00-05:00  1.43  1.28  1.2950    892449  
2021-02-11 00:00:00-05:00  1.69  1.28  1.6478   7198713  
2021-02-12 00:00:00-05:00  2.09  1.33  1.7000  11325167